## Imports all libraries

In [ ]:
import requests
import json

import plotly.express as px
import numpy as np

import pandas as pd
from pandas import json_normalize

## Request the total cases in germany for every day.

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/germany/history/cases"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()

#### Prepare the format

In [ ]:
## Convert to a data frame

cases = pd.json_normalize(data['data'])

print(cases)

#### Start working with your data

In [ ]:
## Check the formats

print(cases.dtypes)

In [ ]:
## Convert date to date format

cases["date"] = pd.to_datetime(cases["date"])

print(cases.dtypes)

In [ ]:
## Sort data by date

cases.sort_values(by='date', ascending = True, inplace=True)

print(cases)

In [ ]:
## Sort data by most cases

cases.sort_values(by='cases', ascending = False, inplace=True)

print(cases)

In [ ]:
## Filter out year 2022

year = 2022

cases_2022 = cases[cases['date'].dt.year == year]

print(cases_2022)

In [ ]:
## Group cases by months

cases_2022_by_month = cases_2022.groupby(cases_2022['date'].dt.strftime('%m %B'))['cases'].sum().sort_index()

print(cases_2022_by_month)

#### Visualize your data

In [ ]:
## Display cases over time

cases['month'] = cases['date'].dt.month

fig = px.bar(cases, x='date', y="cases")

fig.show()

### Request corona stats per state

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()


#### Convert to a Dataframe

In [ ]:
## Convert to a data frame

cases_by_state = pd.DataFrame.from_dict(data['data'], orient='index')

for index, row in cases_by_state.iterrows():
    delta = row["delta"]
    if not pd.isnull(delta):
        for k, v in delta.items():
            key = f'delta_{k}'
            if key not in cases_by_state:
                cases_by_state[key] = np.nan
            cases_by_state.loc[index, key] = v
cases_by_state.drop(columns=["delta"], inplace=True)

for index, row in cases_by_state.iterrows():
    hospitalization = row["hospitalization"]
    if not pd.isnull(hospitalization):
        for k, v in hospitalization.items():
            key = f'hospitalization_{k}'
            if key not in cases_by_state:
                cases_by_state[key] = np.nan
            cases_by_state.loc[index, key] = v
cases_by_state.drop(columns=["hospitalization"], inplace=True)

cases_by_state

#### Visualize your data

### Request hospitalization stats per state

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states/history/hospitalization"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()

print(data)

### Request recovered stats per state

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states/history/recovered"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()

recovered_by_state = pd.DataFrame.from_dict(data['data'], orient='index')

pd.DataFrame(recovered_by_state)